In [108]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os

In [109]:
titanic_df = pd.read_csv("./data/train.csv")

In [110]:
titanic_df["sex_enc"] = np.where(titanic_df["Sex"]=="male", 1, 0)

In [111]:
mean_age = titanic_df["Age"].mean()
titanic_df["new_age"] = titanic_df["Age"].fillna(mean_age)

In [112]:
age_bins = [0, 10, 20, 40, 60, 80, 100]
age_lbl = [10, 20, 40, 60, 80, 100]
titanic_df['age_bin'] = pd.cut(titanic_df['new_age'], bins= age_bins, labels=age_lbl)
titanic_df = pd.get_dummies(titanic_df, columns=["age_bin"])

In [113]:
fare_bins = [0, 50, 100, 200, 600]
fare_lbl = [50, 100, 200, 600]
titanic_df['fare_bin'] = pd.cut(titanic_df['Fare'], bins= fare_bins, labels=fare_lbl)
titanic_df = pd.get_dummies(titanic_df, columns=["fare_bin"])

In [114]:
titanic_df["age_norm"] = (titanic_df["new_age"] - titanic_df["new_age"].mean())/titanic_df["new_age"].std()
titanic_df["sibsp_norm"]=(titanic_df["SibSp"] - titanic_df["SibSp"].min())/(titanic_df["SibSp"].max() - titanic_df["SibSp"].min())
titanic_df["parch_norm"]=(titanic_df["Parch"] - titanic_df["Parch"].min())/(titanic_df["Parch"].max() - titanic_df["Parch"].min())
titanic_df["fare_norm"]=(titanic_df["Fare"] - titanic_df["Fare"].mean())/titanic_df["Fare"].std()

In [115]:
titanic_df["mr_name"] = np.where(titanic_df["Name"].str.contains("Mr\\."), 1, 0)
titanic_df["mrs_name"] = np.where(titanic_df["Name"].str.contains("Mrs\\."), 1, 0)
titanic_df["miss_name"] = np.where(titanic_df["Name"].str.contains("Miss\\."), 1, 0)
titanic_df["master_name"] = np.where(titanic_df["Name"].str.contains("Master\\."), 1, 0)

In [116]:
titanic_df = pd.get_dummies(titanic_df, columns=["Embarked"])
# titanic_df["Pclass"].value_counts()

In [117]:
titanic_df = pd.get_dummies(titanic_df, columns=["Pclass"])

In [146]:
train, val, test = np.split(titanic_df.sample(frac=1), [int(0.8*len(titanic_df)), int(0.9*len(titanic_df))])
print("# training samples: " + str(train.shape))
print("# validation samples: " + str(val.shape))
print("# testing samples: " + str(test.shape))
# feature_list = ["sex_enc", "Parch", "SibSp", "Embarked_C", "Embarked_Q", "Embarked_S"
#                 , "Pclass_1", "Pclass_2", "Pclass_3", "age_bin_20", "age_bin_40", "age_bin_60", "age_bin_80", "age_bin_100"]
# feature_list = ["sex_enc", "parch_norm", "sibsp_norm", "Embarked_C", "Embarked_Q", "Embarked_S"
#                 , "Pclass_1", "Pclass_2", "Pclass_3", "fare_bin_50", "fare_bin_100"
#                 , "fare_bin_200", "fare_bin_600", "mr_name", "mrs_name", "miss_name", "master_name"
#                 , "age_bin_10", "age_bin_20", "age_bin_40", "age_bin_60", "age_bin_80", "age_bin_100"]
feature_list = ["sex_enc", "Embarked_C", "Embarked_Q", "Embarked_S"
                , "Pclass_1", "Pclass_2", "Pclass_3", "mr_name", "mrs_name", "miss_name", "master_name"
                , "age_bin_10", "age_bin_20", "age_bin_40", "age_bin_60", "age_bin_80", "age_bin_100"]

# training samples: (712, 36)
# validation samples: (89, 36)
# testing samples: (90, 36)


In [147]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Survived')
  ds = tf.data.Dataset.from_tensor_slices((dataframe[feature_list], labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [148]:
batch_size = 30
train_ds = df_to_dataset(train, shuffle=False, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [149]:
input_shape = (len(feature_list),)
inputs = tf.keras.layers.Input(shape=input_shape)
x = tf.keras.layers.Dense(32, activation="relu")(inputs)
x = tf.keras.layers.Dropout(0.25)(x)
output = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs, output)
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=["accuracy"])
model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
24/24 [==============================] - 0s 4ms/step - loss: 0.6610 - accuracy: 0.6236 - val_loss: 0.6184 - val_accuracy: 0.7303
Epoch 2/50
24/24 [==============================] - 0s 1ms/step - loss: 0.6078 - accuracy: 0.6994 - val_loss: 0.5719 - val_accuracy: 0.7416
Epoch 3/50
24/24 [==============================] - 0s 1ms/step - loss: 0.5666 - accuracy: 0.7360 - val_loss: 0.5349 - val_accuracy: 0.7528
Epoch 4/50
24/24 [==============================] - 0s 1ms/step - loss: 0.5296 - accuracy: 0.7542 - val_loss: 0.5040 - val_accuracy: 0.7978
Epoch 5/50
24/24 [==============================] - 0s 1ms/step - loss: 0.5146 - accuracy: 0.7640 - val_loss: 0.4816 - val_accuracy: 0.7865
Epoch 6/50
24/24 [==============================] - 0s 1ms/step - loss: 0.4992 - accuracy: 0.7626 - val_loss: 0.4663 - val_accuracy: 0.7978
Epoch 7/50
24/24 [==============================] - 0s 1ms/step - loss: 0.4775 - accuracy: 0.7851 - val_loss: 0.4544 - val_accuracy: 0.7978
Epoch 8/50
24/24 [==

In [150]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

3/3 [==============================] - 0s 998us/step - loss: 0.3563 - accuracy: 0.9000
Loss:  0.35634323954582214
Accuracy:  0.8999999761581421


In [151]:
test_data = pd.read_csv("./data/test.csv")
pred_in = test_data.copy()
pred_in["sex_enc"] = np.where(pred_in["Sex"]=="male", 1, 0)
pred_in["new_age"] = pred_in["Age"].fillna(mean_age)
pred_in = pd.get_dummies(pred_in, columns=["Embarked"])
pred_in = pd.get_dummies(pred_in, columns=["Pclass"])
pred_in['age_bin'] = pd.cut(pred_in['new_age'], bins= age_bins, labels=age_lbl)
pred_in = pd.get_dummies(pred_in, columns=["age_bin"])
pred_in["age_norm"] = (pred_in["new_age"] - titanic_df["new_age"].mean())/titanic_df["new_age"].std()
pred_in["sibsp_norm"]=(pred_in["SibSp"] - titanic_df["SibSp"].min())/(titanic_df["SibSp"].max() - titanic_df["SibSp"].min())
pred_in["parch_norm"]=(pred_in["Parch"] - titanic_df["Parch"].min())/(titanic_df["Parch"].max() - titanic_df["Parch"].min())
pred_in["fare_norm"]=(pred_in["Fare"] - titanic_df["Fare"].mean())/titanic_df["Fare"].std()
pred_in['fare_bin'] = pd.cut(pred_in['Fare'], bins= fare_bins, labels=fare_lbl)
pred_in = pd.get_dummies(pred_in, columns=["fare_bin"])
pred_in["mr_name"] = np.where(pred_in["Name"].str.contains("Mr\\."), 1, 0)
pred_in["mrs_name"] = np.where(pred_in["Name"].str.contains("Mrs\\."), 1, 0)
pred_in["miss_name"] = np.where(pred_in["Name"].str.contains("Miss\\."), 1, 0)
pred_in["master_name"] = np.where(pred_in["Name"].str.contains("Master\\."), 1, 0)

In [152]:
pred_ds = tf.data.Dataset.from_tensor_slices(pred_in[feature_list])
pred_ds = pred_ds.batch(batch_size)

In [153]:
predictions = model.predict(pred_ds)
pred_pd = pd.DataFrame(predictions, columns=["pred_survived"])

In [154]:
pred_pd["Survived"] = np.where(pred_pd["pred_survived"]>0.6, 1, 0)

In [155]:
out = pd.merge(pred_in, pred_pd, left_index=True, right_index=True)

In [156]:
out_pd = out[["PassengerId","Survived"]]

In [157]:
os.makedirs('./data/', exist_ok=True)
out_pd.to_csv('./data/out_1.csv',index=False)